## Database formation and data processing using SQLALchemy

In [1]:
#Import dependencies
import pandas as pd
import sqlite3 as sqlite
from datetime import datetime as dt
import numpy as np
from sqlalchemy import create_engine

#Path name files to datasets for cleaning and processing.  Jeopardy CSV is from Kaggle, Contestants and Locations
# are from github/anuparna/jeopardy.  jeaopardy_archive is the flat file Geetha found
path1 = ('./Resources/jeopardy_archive_data.csv')
path2 = ('./Resources/contestants.csv')
path3 = ('./Resources/locations.csv')
path4 = ('./Resources/final_results.csv')
path5 = ('./Resources/questions.csv')
path6 = ('./Resources/question_count_NLTK.csv')
path7 = ('./Resources/answer_count_NLTK.csv')
path8 = ('./Resources/category_count_NLTK.csv')






In [2]:
#Connector to sqlite3.  Creates Jeopardy database
conn = sqlite.connect('jeopardy_db.sqlite')

In [3]:
from numpy import datetime64

#Read and format csv files from outside sources into dataframe.  

df1 = pd.read_csv(path1)
df2 = pd.read_csv(path2)
df3 = pd.read_csv(path3)
df4 = pd.read_csv(path4)
df5 = pd.read_csv(path5)

jeopardy_df1 = pd.DataFrame(df1)
jeopardy_df1['date'] = pd.to_datetime(jeopardy_df1['date'], errors='coerce')
jeopardy_df1['Year'] = jeopardy_df1['date'].dt.strftime('%Y')
jeopardy_df1 = pd.DataFrame(jeopardy_df1).rename(columns={'full_name': 'Name'})

#Removed extra text from jeopardy_archive data to join fields on "show_info"

jeopardy_df1['show_info'] = jeopardy_df1['show_info'].str.replace(r'\D','')
jeopardy_df1['show_info'] = jeopardy_df1['show_info'].astype('int64')
num=100000000
jeopardy_df1['show_info'] =(jeopardy_df1['show_info']/num).round(0)
jeopardy_df1['show_info'] = jeopardy_df1['show_info'].astype('int64')

jeopardy_df1 = jeopardy_df1.drop(['nickname', 'player_details', 'archive_info'], axis=1)

jeopardy_df2 = pd.DataFrame(df2)
jeopardy_df2['Name'] = jeopardy_df2['player_first_name'].map(str) + ' ' + jeopardy_df2['player_last_name'].map(str)
jeopardy_df2 = jeopardy_df2.drop(columns = ['player_first_name', 'player_last_name'])
jeopardy_df2 = jeopardy_df2.drop_duplicates(subset = 'Name')

jeopardy_df3 = pd.DataFrame(df3)
jeopardy_df4 = pd.DataFrame(df4)
jeopardy_df5 = pd.read_csv(path5)



/var/folders/xw/z2rhr7zj1kvg3zdq580t0_340000gp/T/ipykernel_72984/1397060734.py:18: FutureWarning: The default value of regex will change from True to False in a future version.
  jeopardy_df1['show_info'] = jeopardy_df1['show_info'].str.replace(r'\D','')


In [4]:
#Read datasets into Sqlite table

jeopardy_df1.to_sql('jeopardy_archvie_tbl', conn, if_exists='replace', index = False)
jeopardy_df2.to_sql('jeopardy_contestants_tbl', conn, if_exists='replace', index = False)
jeopardy_df3.to_sql('jeopardy_seat_locations_tbl', conn, if_exists='replace', index = False)
jeopardy_df4.to_sql('jeopardy_final_results_tbl', conn, if_exists='replace', index = False)
jeopardy_df5.to_sql('jeopardy_questions_tbl', conn, if_exists='replace', index = False)
conn.commit()


In [5]:
#Create an instance of database in SQLAlchemy
engine = create_engine('sqlite:///./jeopardy_db.sqlite')

In [6]:
#SQL query to produce contestant data
ContestantSQL = pd.read_sql('''
SELECT A.Name, B.player_id, A.occupation, B.hometown_city, B.hometown_state, A.Year, A.show_info  
FROM jeopardy_archvie_tbl A
FULL OUTER JOIN  jeopardy_contestants_tbl B ON A.Name = B.Name
WHERE Year >=2014
'''
, engine.connect())

In [7]:
#Explore the data to find NanNs of contestants not listed in jeopardy_contestants_tbl
display(ContestantSQL.sort_values('show_info').head())
display(ContestantSQL.sort_values('show_info').tail())

,Name,player_id,occupation,hometown_city,hometown_state,Year,show_info
3908,John Coulter,8838.0,a creative director,Henderson,Nevada,2014,6743
3907,Bridget O'Donnell,8839.0,a law enforcement specialist,Burlington,Vermont,2014,6743
3906,Jerry Slowik,8832.0,a writer,Arlington Heights,Illinois,2014,6743
3903,Jerry Slowik,8832.0,a writer,Arlington Heights,Illinois,2014,6744
3905,Rachel Samberg,8840.0,an academic tutor and performer,New York,New York,2014,6744


,Name,player_id,occupation,hometown_city,hometown_state,Year,show_info
4,Eric Kaplan,NaN,a retired OB/GYN physician,None,None,2019,8044
3,Jason Zuffranieri,NaN,a math teacher,None,None,2019,8044
2,Michael Riggs,NaN,an educational therapist,None,None,2019,8045
1,Maggie Lehrman,NaN,an editor and writer,None,None,2019,8045
0,Jason Zuffranieri,NaN,a math teacher,None,None,2019,8045


In [8]:
ContestantSQL.info()
ContestantSQL.to_sql('jeopardy_ContestantSQL_tbl', conn, if_exists='replace', index = False)
conn.commit()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3909 entries, 0 to 3908
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Name            3909 non-null   object 
 1   player_id       2358 non-null   float64
 2   occupation      3905 non-null   object 
 3   hometown_city   2332 non-null   object 
 4   hometown_state  2332 non-null   object 
 5   Year            3909 non-null   object 
 6   show_info       3909 non-null   int64  
dtypes: float64(1), int64(1), object(5)
memory usage: 213.9+ KB


In [9]:
Game_ID = pd.read_sql('''
SELECT DISTINCT B.game_id, C.question_text, C.answer, C. category
FROM  jeopardy_ContestantSQL_tbl A
INNER JOIN jeopardy_seat_locations_tbl B ON A.player_id = B.player_id
INNER JOIN jeopardy_questions_tbl C ON B.game_id = C.game_id ''' 
, engine.connect())

In [10]:
display(Game_ID.sort_values('game_id').head(10))
display(Game_ID.sort_values('game_id').tail())

,game_id,question_text,answer,category
222,901,This Steelers wide receiver didn't need 57 rea...,Hines Ward,SUPER BOWL MVPs
200,901,"Elton's 1982 song ""Empty Garden (Hey Hey Johnn...",John Lennon,ELTON JOHN
199,901,Completes the apt title of Elton's 1973 album ...,"""The Piano Player""",ELTON JOHN
198,901,"A 1982 Elton John song says ""Baby's got"" these...","""Blue Eyes""",ELTON JOHN
197,901,This term for a freshwater crocodile of India ...,a mugger,CROCODILE ROCK
196,901,The god Sobek of this ancient culture was ofte...,Ancient Egypt,CROCODILE ROCK
195,901,The American crocodile is very rare & can only...,Florida,CROCODILE ROCK
194,901,Crocodiles are unique among reptiles in that t...,the heart,CROCODILE ROCK
193,901,Crikey! This man's made quite a career as the...,Steve Irwin,CROCODILE ROCK
192,901,To cause annoyance or ire,rile,CIRCLE OF LIFE


,game_id,question_text,answer,category
5133,5637,"Keep, parapet, moat",a castle,PARTS OF THE WHOLE
5134,5637,"Phone clamp, extendable monopod, shutter button",a selfie stick,PARTS OF THE WHOLE
5135,5637,"Style, stigma, stamen",the flower,PARTS OF THE WHOLE
5121,5637,Chris Mullin,basketball,HALL OF FAMERS BY SPORT
5108,5637,"On April 16, 1963 he wrote, ""I am in Birmingha...",Martin Luther King Jr.,20th CENTURY AMERICA


In [11]:
Game_ID.dropna()

,game_id,question_text,answer,category
0,3901,"A longshot who wins, or a ""Star Wars"" comic bo...",a dark horse,"""DAR"""
1,3901,"A term of endearment, like for my Clementine",darling,"""DAR"""
2,3901,It's the Turkish strait seen here,the Dardanelles,"""DAR"""
3,3901,The inner bull's eye is commonly worth 50 poin...,darts,"""DAR"""
4,3901,This capital of Australia's Northern Territory...,Darwin,"""DAR"""
...,...,...,...,...
49890,4385,Alexander Solzhenitsyn spent years in the Gula...,Stalin,WRITERS IN PRISON
49891,4385,"An author, statesman & saint, he was jailed in...",Thomas More,WRITERS IN PRISON
49892,4385,He spent 30 days in jail for vagrancy in 1894 ...,Jack London,WRITERS IN PRISON
49893,4385,"The ""idiot""! He got mixed up with the Petrashe...",Dostoyevsky,WRITERS IN PRISON


In [12]:
Game_ID.to_csv('QuestionsSQL.csv')

In [13]:
NLTK_df1 = pd.read_csv(path6)
NLTK_df2 = pd.read_csv(path7)
NLTK_df3 = pd.read_csv(path8)

NLTK_df1.drop(columns ='Unnamed: 0' ).to_sql('Questions_count_NLTK', conn, if_exists='replace', index = False)
NLTK_df2.drop(columns ='Unnamed: 0' ).to_sql('Answer_count_NLTK', conn, if_exists='replace', index = False)
NLTK_df3.drop(columns ='Unnamed: 0' ).to_sql('Category_count_NLTK', conn, if_exists='replace', index = False)


4944

In [14]:
words_SQL = pd.read_sql('''
SELECT A.Word AS Question_Word, A.Count AS Question_Count, B.Word AS Answer_Word, B.Count AS Answer_Count, C.Word AS Category_Word, C.Count AS Category_Count
FROM Questions_count_NLTK A
RIGHT JOIN Answer_count_NLTK B ON A.Word = B.Word
RIGHT JOIN Category_count_NLTK C ON B.Word = C.Word'''
                        , engine.connect())
conn.commit()


In [15]:
words_SQL.dropna().sort_values(by = ['Category_Count', 'Question_Count', 'Answer_Count'], ascending = [False, False, False]).info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3318 entries, 44 to 2367
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Question_Word   3318 non-null   object 
 1   Question_Count  3318 non-null   float64
 2   Answer_Word     3318 non-null   object 
 3   Answer_Count    3318 non-null   float64
 4   Category_Word   3318 non-null   object 
 5   Category_Count  3318 non-null   int64  
dtypes: float64(2), int64(1), object(3)
memory usage: 181.5+ KB


In [16]:
words_SQL.to_csv('wordsSQL.csv')

In [17]:
EarningsSQL = pd.read_sql('''
SELECT A.Name, A.occupation, B.hometown_city, B.hometown_state, A.Year, A.show_info, A.final_score  
FROM jeopardy_archvie_tbl A
FULL OUTER JOIN  jeopardy_contestants_tbl B ON A.Name = B.Name
WHERE Year >=2014
''', engine.connect())
conn.commit()


display(EarningsSQL.head(5))
display(EarningsSQL.tail(5))

,Name,occupation,hometown_city,hometown_state,Year,show_info,final_score
0,Jason Zuffranieri,a math teacher,None,None,2019,8045,"$27,600"
1,Maggie Lehrman,an editor and writer,None,None,2019,8045,$0
2,Michael Riggs,an educational therapist,None,None,2019,8045,$2
3,Jason Zuffranieri,a math teacher,None,None,2019,8044,"$4,400"
4,Eric Kaplan,a retired OB/GYN physician,None,None,2019,8044,$0


,Name,occupation,hometown_city,hometown_state,Year,show_info,final_score
3904,Sarah Olson,a project coordinator,El Cerrito,California,2014,6744,"$10,600"
3905,Rachel Samberg,an academic tutor and performer,New York,New York,2014,6744,"$7,800"
3906,Jerry Slowik,a writer,Arlington Heights,Illinois,2014,6743,"$23,000"
3907,Bridget O'Donnell,a law enforcement specialist,Burlington,Vermont,2014,6743,"$5,800"
3908,John Coulter,a creative director,Henderson,Nevada,2014,6743,$799


In [18]:
EarningsSQL.to_csv('EarningsSQL.csv')